In [ ]:
# setup
import pandas as pd
import ast
import string
from google.colab import drive
import re
from datetime import datetime
# drive.mount('/content/drive')

# ratings_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/Raw_datasets/Books_rating.csv")
# ratings = ratings_raw[['Title', 'Id']].drop_duplicates(subset = 'Title', keep = 'first') # keep only 1 ID per title
# ratings = ratings.dropna()
# books = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/Raw_datasets/books_data.csv")
# enrichment = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/enrichments/amazonbooks_enrichment_withratings_50k.csv")

In [ ]:
# data cleaning
bm = books.merge(ratings[['Title', 'Id']], on='Title', how='left') # add ID info
bm['publishedDate'] = bm['publishedDate'].str[:4] # only take year
bm['authors'].tolist()
def safe_eval(val):
    if pd.isna(val):
        return []
    return ast.literal_eval(val)
bm['authors'] = bm['authors'].apply(safe_eval)
bm['authors'] = bm['authors'].apply(lambda x: [author.strip('"') for author in x])
bm['authors'] = bm['authors'].apply(lambda x: None if isinstance(x, list) and len(x) == 0 else x)

bm['categories'] = bm['categories'].apply(safe_eval)
bm['categories'] = bm['categories'].apply(lambda x: [category.strip('"') for category in x])
bm['categories'] = bm['categories'].apply(lambda x: None if isinstance(x, list) and len(x) == 0 else x)
bm['categories'] = bm['categories'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [ ]:
# mapping table of author to book ID (accounts for >1 author/book)
authors_mapping = bm.explode('authors', ignore_index=True)
cols = ['Title', 'Id', 'authors']
authors_mapping = authors_mapping[cols]
authors_mapping = authors_mapping.rename(columns={"Title": "title", "Id": "id"})
authors_mapping = authors_mapping.dropna(subset=['title', 'id', 'authors'])
authors_mapping = authors_mapping.drop_duplicates(keep="first")
#authors_mapping.to_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/authors_mapping_modified.csv", index=False, header = True)

In [ ]:
bm = bm.drop(columns = 'authors')
bm = bm.merge(enrichment, left_on='Title', right_on = 'name', how='left')

# rename and reorder columns, save to CSV
bm = bm.drop(columns = 'name')
bm = bm.rename(columns={"Title": "title", "Id": "id"}, errors="raise")
bm = bm.rename(columns={"previewLink": "preview_link",
                        "publishedDate": "published_date",
                        "infoLink": "info_link",
                        "ratingsCount": "ratings_count"}, errors="raise")
new_col_order = ["id", "title", "description", "image", "preview_link",
                 "publisher", "published_date", "info_link", "categories",
                 "ratings_count", "city_set_in", "country_set_in",
                 "period_set_in", "film_name", "classification"]
bm = bm[new_col_order]
# drop rows with NULL values for id
bm = bm.dropna(subset=['id'])
bm = bm.where(pd.notnull(bm), None)

In [ ]:
def extract_periods(value):
    if isinstance(value, str):
        # Handle "various time periods" (or similar unrecognized text)
        if 'various time periods' in value.lower():
            return None, None

        # Handle "Contemporary" (let's assume contemporary is 2000 to current year)
        if 'contemporary' in value.lower():
            current_year = datetime.now().year
            return 2000, current_year  # Assuming "Contemporary" means 2000-present

        # Handle "c. X BC - c. Y BC" by processing the 'c.' and BCE years
        if 'c.' in value.lower() and 'BC' in value.upper():
            value = value.replace('c.', '').strip()  # Remove 'c.'
            parts = re.split(r'\s*-\s*', value)
            start, end = None, None

            if len(parts) == 2:  # Ensure we have a range
                # Handle the start date
                start_match = re.search(r'(\d+)\s*BC', parts[0], re.IGNORECASE)
                if start_match:
                    start = -int(start_match.group(1))  # BCE years are negative

                # Handle the end date
                end_match = re.search(r'(\d+)\s*BC', parts[1], re.IGNORECASE)
                if end_match:
                    end = -int(end_match.group(1))  # BCE years are negative

            return start, end

        # Handle "about X - Y" by removing "about" and processing the numbers
        if 'about' in value.lower():
            value = value.lower().replace('about', '').strip()
            parts = re.split(r'\s*-\s*', value)
            if len(parts) == 2:
                start = int(parts[0].replace(',', ''))  # Remove commas and convert to int
                end = int(parts[1].replace(',', ''))    # Same for the second number
                return start, end

        # Handle '19th century'
        elif re.search(r'\d{1,2}(st|nd|rd|th) century', value, re.IGNORECASE):
            match = re.search(r'(\d{1,2})', value)
            if match:
                century = int(match.group(1))
                start = (century - 1) * 100
                end = start + 99
                return start, end

        # Handle a range like '1930-1940'
        elif ' - ' in value:
            parts = value.split(' - ')
            if len(parts) == 2 and all(part.isdigit() for part in parts):
                start = int(parts[0])
                end = int(parts[1])
                return min(start, end), max(start, end)  # Ensure we return a valid range

        # Handle centuries like '19th century'
        elif re.search(r'\d{1,2}(st|nd|rd|th) century', value, re.IGNORECASE):
            match = re.search(r'(\d{1,2})', value)
            if match:
                century = int(match.group(1))
                start = (century - 1) * 100
                end = start + 99
                return start, end

        # Handle ranges with BCE/CE (e.g., 'c. 1200 BCE - 70 CE')
        if re.search(r'BCE|CE', value, re.IGNORECASE):
            value = value.replace("c.", "").strip()  # Remove 'c.' if present
            parts = re.split(r'\s*-\s*', value)
            start, end = None, None

            if len(parts) == 2:  # Ensure we have a range
                # Handle the start date
                start_match = re.search(r'(\d+)\s*(BCE|CE)', parts[0], re.IGNORECASE)
                if start_match:
                    start = int(start_match.group(1))
                    if start_match.group(2).upper() == "BCE":
                        start = -start  # BCE years are negative

                # Handle the end date
                end_match = re.search(r'(\d+)\s*(BCE|CE)', parts[1], re.IGNORECASE)
                if end_match:
                    end = int(end_match.group(1))
                    if end_match.group(2).upper() == "BCE":
                        end = -end  # BCE years are negative

            return start, end

        # Handle single years like '1899' or '1200 BCE'
        elif re.search(r'\d+\s*(BCE|CE)', value, re.IGNORECASE):
            match = re.search(r'(\d+)\s*(BCE|CE)', value, re.IGNORECASE)
            year = int(match.group(1))
            if match.group(2).upper() == "BCE":
                year = -year  # BCE years are negative
            return year, year

        elif value.isdigit():  # Single numeric year
            year = int(value)
            return year, year

        # If the value is unrecognized text, print it and return None
        return None, None

    # If the value is None or empty string, return None
    return None, None

bm[['time_period_start', 'time_period_end']] = bm['period_set_in'].apply(extract_periods).apply(pd.Series)

In [ ]:
cities = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/11-24_updated_cities.csv")
cities.loc[cities['country_name'] == 'Namibia', 'country_iso'] = 'NA' # fixing Namibia country ISO being NA
cities = cities.dropna(subset = 'city')
cities = cities.rename(columns={"id": "setting_id"})
#cities.to_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/new_cities_final.csv", index=False)

In [ ]:
!pip install dataprep

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask[array,dataframe,delayed] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of dask[array,dataframe,delayed] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of flask to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This

In [ ]:
# keep only first city and first country
bm['first_city'] = bm['city_set_in'].str.lower().str.split(',').str[0]
bm['first_country'] = bm['country_set_in'].str.lower().str.split(',').str[0]
bm = bm.rename(columns={"first_city": "city", "first_country": "country"})

# create country_iso column
from dataprep.clean import clean_country
bm_iso = clean_country(bm,"country",output_format="alpha-2",fuzzy_dist=5)

NameError: name 'bm' is not defined

In [ ]:
bm_w_id = pd.merge(bm_iso, cities[['setting_id', 'city', 'country_iso']],
                     left_on=['city_set_in', 'country_clean'],
                     right_on=['city', 'country_iso'],
                     how='left')

cols_to_keep = ['id', 'title', 'description', 'image', 'preview_link', 'publisher',
                'published_date', 'info_link', 'categories', 'ratings_count',
                'film_name', 'classification', 'time_period_start',
                'time_period_end', 'setting_id']

bm_w_id = bm_w_id[cols_to_keep]
bm_w_id = bm_w_id.drop_duplicates(subset='id', keep='first')

In [ ]:
bm_w_id['categories'] = bm_w_id['categories'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
bm_w_id['categories'] = bm_w_id['categories'].str.lower()


In [ ]:
import csv
print(bm_w_id.head())
#print(bm_w_id['categories']).tolist()
bm_w_id.to_csv('/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/amazon_books_final.csv', index=False, quoting=csv.QUOTE_MINIMAL)

           id                                              title  \
0  1882931173                     Its Only Art If Its Well Hung!   
1  0826414346                           Dr. Seuss: American Icon   
2  0829814000              Wonderful Worship in Smaller Churches   
3  0595344550                      Whispers of the Wicked Saints   
4  0253338352  Nation Dance: Religion, Identity and Cultural ...   

                                         description  \
0                                               None   
1  Philip Nel takes a fascinating look into the k...   
2  This resource includes twelve principles in un...   
3  Julia Thomas finds her life spinning out of co...   
4                                               None   

                                               image  \
0  http://books.google.com/books/content?id=DykPA...   
1  http://books.google.com/books/content?id=IjvHQ...   
2  http://books.google.com/books/content?id=2tsDA...   
3  http://books.google.com/boo

In [ ]:
### Clean ratings ###
bm_ids = bm_w_id['id'].tolist()

ratings_cleaned = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/cleaned_Books_rating.csv")
ratings_nona = ratings_cleaned.dropna(subset=['user_id']) # remove rows with NA values for user_id
# for duplicated user + book id combos, replace helpfulness with average for book + user and drop duplicates
ratings_nona['review_helpfulness'] = ratings_nona.groupby(['user_id', 'book_id'])['review_helpfulness'].transform('mean')
ratings_nona = ratings_nona.drop_duplicates(subset=['user_id', 'book_id'])
# drop rows with a book that isn't in Amazon books
ratings_nona = ratings_nona[ratings_nona['book_id'].isin(bm_ids)]

ratings_nona.to_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/ratings_w_id_modified.csv", index=False)

<ipython-input-150-51fe8e520211>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_nona['review_helpfulness'] = ratings_nona.groupby(['user_id', 'book_id'])['review_helpfulness'].transform('mean')


All book IDs from ratings are present in bm_w_id.


##BX Dataset

In [ ]:
bx_users = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/bx_users_new_location.csv")
bx_users = bx_users.replace({pd.NA: None, pd.NaT: None, float('nan'): None})
# check if location ID is in cities dataset
rows_with_invalid_location = bx_users[~bx_users['location_id'].isin(cities['setting_id'])]
bx_users.head()
bx_users.to_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/bx_users_new_location_modified.csv", index=False)

In [ ]:
#!pip install dataprep

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 60.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask[array,dataframe,delayed] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of dask[array,dataframe,delayed] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of flask to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This

In [ ]:
import re
from datetime import datetime
# add enrichment data to bx books
bx_books = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/bx_books.csv")
bx_enrichment = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/enrichments/bx_enrichment_ratingsGT2_numratings>2.csv")
bx_books = bx_books.drop(columns=['film_name', 'classification', 'time_period_start', 'time_period_end'])
enrichment_noduplicates = bx_enrichment.drop_duplicates(subset=['name', 'authors'], keep='first')
bx_enriched = bx_books.merge(enrichment_noduplicates,
                             left_on=['title', 'author'],
                             right_on=['name', 'authors'],
                             how='left').drop(columns=['name', 'authors'])

def extract_periods(value):
    if isinstance(value, str):
        # Handle "various time periods" (or similar unrecognized text)
        if 'various time periods' in value.lower():
            return None, None

        # Handle "Contemporary" (let's assume contemporary is 2000 to current year)
        if 'contemporary' in value.lower():
            current_year = datetime.now().year
            return 2000, current_year  # Assuming "Contemporary" means 2000-present

        # Handle "c. X BC - c. Y BC" by processing the 'c.' and BCE years
        if 'c.' in value.lower() and 'BC' in value.upper():
            value = value.replace('c.', '').strip()  # Remove 'c.'
            parts = re.split(r'\s*-\s*', value)
            start, end = None, None

            if len(parts) == 2:  # Ensure we have a range
                # Handle the start date
                start_match = re.search(r'(\d+)\s*BC', parts[0], re.IGNORECASE)
                if start_match:
                    start = -int(start_match.group(1))  # BCE years are negative

                # Handle the end date
                end_match = re.search(r'(\d+)\s*BC', parts[1], re.IGNORECASE)
                if end_match:
                    end = -int(end_match.group(1))  # BCE years are negative

            return start, end

        # Handle "about X - Y" by removing "about" and processing the numbers
        if 'about' in value.lower():
            value = value.lower().replace('about', '').strip()
            parts = re.split(r'\s*-\s*', value)
            if len(parts) == 2:
                start = int(parts[0].replace(',', ''))  # Remove commas and convert to int
                end = int(parts[1].replace(',', ''))    # Same for the second number
                return start, end

        # Handle '19th century'
        elif re.search(r'\d{1,2}(st|nd|rd|th) century', value, re.IGNORECASE):
            match = re.search(r'(\d{1,2})', value)
            if match:
                century = int(match.group(1))
                start = (century - 1) * 100
                end = start + 99
                return start, end

        # Handle a range like '1930-1940'
        elif ' - ' in value:
            parts = value.split(' - ')
            if len(parts) == 2 and all(part.isdigit() for part in parts):
                start = int(parts[0])
                end = int(parts[1])
                return min(start, end), max(start, end)  # Ensure we return a valid range

        # Handle centuries like '19th century'
        elif re.search(r'\d{1,2}(st|nd|rd|th) century', value, re.IGNORECASE):
            match = re.search(r'(\d{1,2})', value)
            if match:
                century = int(match.group(1))
                start = (century - 1) * 100
                end = start + 99
                return start, end

        # Handle ranges with BCE/CE (e.g., 'c. 1200 BCE - 70 CE')
        if re.search(r'BCE|CE', value, re.IGNORECASE):
            value = value.replace("c.", "").strip()  # Remove 'c.' if present
            parts = re.split(r'\s*-\s*', value)
            start, end = None, None

            if len(parts) == 2:  # Ensure we have a range
                # Handle the start date
                start_match = re.search(r'(\d+)\s*(BCE|CE)', parts[0], re.IGNORECASE)
                if start_match:
                    start = int(start_match.group(1))
                    if start_match.group(2).upper() == "BCE":
                        start = -start  # BCE years are negative

                # Handle the end date
                end_match = re.search(r'(\d+)\s*(BCE|CE)', parts[1], re.IGNORECASE)
                if end_match:
                    end = int(end_match.group(1))
                    if end_match.group(2).upper() == "BCE":
                        end = -end  # BCE years are negative

            return start, end

        # Handle single years like '1899' or '1200 BCE'
        elif re.search(r'\d+\s*(BCE|CE)', value, re.IGNORECASE):
            match = re.search(r'(\d+)\s*(BCE|CE)', value, re.IGNORECASE)
            year = int(match.group(1))
            if match.group(2).upper() == "BCE":
                year = -year  # BCE years are negative
            return year, year

        elif value.isdigit():  # Single numeric year
            year = int(value)
            return year, year

        # If the value is unrecognized text, print it and return None
        return None, None

    # If the value is None or empty string, return None
    return None, None

bx_enriched[['time_period_start', 'time_period_end']] = bx_enriched['period_set_in'].apply(extract_periods).apply(pd.Series)
bx_enriched = bx_enriched.drop(columns=['period_set_in'])

In [ ]:
# Specific case handling for "New York, Toronto"
mask = bx_enriched["city_set_in"] == "New York, Toronto"
bx_enriched.loc[mask, "city_set_in"] = "New York"
bx_enriched.loc[mask, "country_set_in"] = "United States"

In [ ]:
# Fixing city_set_in and country_set_in to keep only the first value if multiple
bx_enriched["city_set_in"] = bx_enriched["city_set_in"].str.split(",").str[0].str.strip()
bx_enriched["country_set_in"] = bx_enriched["country_set_in"].str.split(",").str[0].str.strip()

In [ ]:
from dataprep.clean import clean_country
bx_enriched = clean_country(bx_enriched,"country_set_in",output_format="alpha-2",fuzzy_dist=5)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7365: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


Country Cleaning Report:
	43912 values cleaned (16.18%)
	2 values unable to be parsed (0.0%), set to NaN
Result contains 43912 (16.18%) values in the correct format and 227467 null values (83.82%)


In [ ]:
# merge with cities to get location set in
bx_enriched_merged = pd.merge(bx_enriched, cities[['setting_id', 'city', 'country_iso']],
                     left_on=['city_set_in', 'country_set_in_clean'],
                     right_on=['city', 'country_iso'],
                     how='left')

In [ ]:
# drop set_in_location column
bx_enriched_merged = bx_enriched_merged.drop(columns=['set_in_location'])

In [ ]:
# rename setting_id to set_in_location
bx_enriched_merged = bx_enriched_merged.rename(columns={"setting_id": "set_in_location"})

In [ ]:
to_keep = ['isbn', 'title', 'author', 'year_published', 'publisher', 'image_url_s',
           'image_url_m', 'image_url_l', 'summary', 'language', 'category',
           'film_name', 'classification', 'set_in_location', 'time_period_start',
           'time_period_end']
bx_enriched_merged = bx_enriched_merged[to_keep]
bx_enriched_merged = bx_enriched_merged.drop_duplicates(subset=['isbn'], keep='first')

In [ ]:
# bx_enriched = bx_books
bx_reviews = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/bx_reviews.csv")
bx_users_ids = bx_users['id'].tolist()
bx_books_isbns = bx_enriched_merged['isbn'].tolist()
bx_reviews = bx_reviews[bx_reviews['user_id'].isin(bx_users_ids)]
bx_reviews = bx_reviews[bx_reviews['isbn'].isin(bx_books_isbns)]

##Check foreign key constraints

In [ ]:
# bx_books
rows_with_invalid_location = bx_enriched_merged[~bx_enriched_merged['set_in_location'].isin(cities['setting_id'])]
rows_with_invalid_location['set_in_location'].unique()

array([nan])

In [ ]:
#bx_users
rows_with_invalid_location = bx_users[~bx_users['location_id'].isin(cities['setting_id'])]
print(rows_with_invalid_location['location_id'].unique())

#bx_ratings
rows_with_invalid_book = bx_reviews[~bx_reviews['isbn'].isin(bx_enriched_merged['isbn'])]
print(rows_with_invalid_book['isbn'].unique())

[None]
[]


##Replace NaN values with None values

In [ ]:
bx_reviews = bx_reviews.replace({pd.NA: None, pd.NaT: None, float('nan'): None})
bx_enriched_merged = bx_enriched_merged.replace({pd.NA: None, pd.NaT: None, float('nan'): None})
bx_users = bx_users.replace({pd.NA: None, pd.NaT: None, float('nan'): None})

##Save dataframes

In [ ]:
bx_enriched_merged.to_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/bx_books_final_final.csv", index=False)
bx_reviews.to_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/bx_reviews_final.csv", index=False)
bx_users.to_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/bx_users_final.csv", index=False)